Pyspark mllib packages are obsolete (there were used for RDD) and Pyspark.ml packages are used for new type (SQL(DATA FRAME) all)

In [2]:
from __future__ import print_function
import pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

'C:\\spark-3.0.0-bin-hadoop3.2\\'

In [3]:
spark = SparkSession.builder.appName("Pyspark ML Algorithms").getOrCreate()

24/09/27 00:34:29 WARN Utils: Your hostname, wtc12 resolves to a loopback address: 127.0.1.1; using 10.64.26.83 instead (on interface enp3s0)
24/09/27 00:34:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/27 00:34:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
dataframe = spark.read.csv("/spark_tutorial/admission_data.csv",header=True)

In [9]:
type (dataframe)

pyspark.sql.dataframe.DataFrame

In [10]:
dataframe.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|      308|        101|                2|  3|   4| 7.9|       0|            0.68|
|      302|        102|                1|  2| 1.5|   8|       0|             0.5|
|      323|     

In [12]:
dataframe.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



All of the coloums data types are string but in machine learning we neeed them in float or int

In [49]:
from pyspark.sql.functions import col
for c in dataframe.columns:
    print((c))
print("\nafter applying coloumn function")
for c in dataframe.columns:
    print(col(c))    
new_dataframe = dataframe.select(*(col(c).cast("float").alias(c) for c in dataframe.columns))

GRE Score
TOEFL Score
University Rating
SOP
LOR 
CGPA
Research
Chance of Admit 

after applying coloumn function
Column<'GRE Score'>
Column<'TOEFL Score'>
Column<'University Rating'>
Column<'SOP'>
Column<'LOR '>
Column<'CGPA'>
Column<'Research'>
Column<'Chance of Admit '>


In [21]:
dataframe.select(*(col(c) for c in dataframe.columns)).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|      308|        101|                2|  3|   4| 7.9|       0|            0.68|
|      302|        102|                1|  2| 1.5|   8|       0|             0.5|
|      323|     

In [23]:
dataframe.select(*(col(c).cast("float") for c in dataframe.columns)).printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



In [ ]:
dataframe.select(*(col(c).cast("float") for c in dataframe.columns)).printSchema()

select(*[col("column1"), col("column2")]) is similar to select(col("column1"), col("column2"))

In [30]:
dataframe.select(col('GRE Score').cast("float")).printSchema()

root
 |-- GRE Score: float (nullable = true)



In [26]:
dataframe.withColumn("GRE Score", col("GRE Score").cast("float")).printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [41]:
new_dataframe.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



In [32]:
from pyspark.sql.functions import col, count, isnan, when #when is equivelent to where in sql

In [33]:
for c in new_dataframe.columns:
    print(c)

GRE Score
TOEFL Score
University Rating
SOP
LOR 
CGPA
Research
Chance of Admit 


In [42]:
new_dataframe.show(4)

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
+---------+-----------+-----------------+---+----+----+--------+----------------+
only showing top 4 rows



In [50]:
#checking for null ir nan type values in our columns
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|        0|          0|                0|  0|   0|   0|       0|               0|
+---------+-----------+-----------------+---+----+----+--------+----------------+



In [51]:
#checking for null ir nan type values in our columns
new_dataframe.select([ count (when (col (c).isNull(), c) ) for c in new_dataframe.columns]).show()

+-------------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------------------+-------------------------------------------+---------------------------------------------+---------------------------------------------+-----------------------------------------------------+---------------------------------------------------------------------+
|count(CASE WHEN (GRE Score IS NULL) THEN GRE Score END)|count(CASE WHEN (TOEFL Score IS NULL) THEN TOEFL Score END)|count(CASE WHEN (University Rating IS NULL) THEN University Rating END)|count(CASE WHEN (SOP IS NULL) THEN SOP END)|count(CASE WHEN (LOR  IS NULL) THEN LOR  END)|count(CASE WHEN (CGPA IS NULL) THEN CGPA END)|count(CASE WHEN (Research IS NULL) THEN Research END)|count(CASE WHEN (Chance of Admit  IS NULL) THEN Chance of Admit  END)|
+-------------------------------------------------------+-------------------------------------------

In [52]:
from pyspark.ml.feature import Imputer

In [54]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_dataframe)

imputed_data = model.transform(new_dataframe)

In [55]:
imputed_data

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR : float, CGPA: float, Research: float, Chance of Admit : float]

In [56]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|        0|          0|                0|  0|   0|   0|       0|               0|
+---------+-----------+-----------------+---+----+----+--------+----------------+



In [73]:
features = imputed_data.drop('Chance of Admit ')

In [74]:
features.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research']

In [75]:
#let's assemble our features together using vectorAssembler


# it will take all columns and combine them like a vector (1,3,4,5)

assembler = VectorAssembler( inputCols=features.columns,outputCol="features")

#we have created new coloumn features where we will have our vector

In [76]:
output = assembler.transform(imputed_data)

In [77]:
output.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |            features|
+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|[324.0,107.0,4.0,...|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|[316.0,104.0,3.0,...|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|              5.0|4.5| 3.0|9.34|     1.0|             0.9|[330.0,115.0,5.0,...|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|         

In [78]:
output.select("features").toPandas()

,features
0,"[337.0, 118.0, 4.0, 4.5, 4.5, 9.64999961853027..."
1,"[324.0, 107.0, 4.0, 4.0, 4.5, 8.86999988555908..."
2,"[316.0, 104.0, 3.0, 3.0, 3.5, 8.0, 1.0]"
3,"[322.0, 110.0, 3.0, 3.5, 2.5, 8.67000007629394..."
4,"[314.0, 103.0, 2.0, 2.0, 3.0, 8.21000003814697..."
...,...
495,"[332.0, 108.0, 5.0, 4.5, 4.0, 9.02000045776367..."
496,"[337.0, 117.0, 5.0, 5.0, 5.0, 9.86999988555908..."
497,"[330.0, 120.0, 5.0, 4.5, 5.0, 9.5600004196167,..."
498,"[312.0, 103.0, 4.0, 4.0, 5.0, 8.43000030517578..."


## Linear Regressor

In [79]:
output= output.select("features", "Chance of Admit ")

In [80]:
output = assembler.transform(imputed_data)

In [81]:
output= output.select("features", "Chance of Admit ")

In [82]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [83]:
train_df.show()
test_df.show()

+--------------------+----------------+
|            features|Chance of Admit |
+--------------------+----------------+
|[290.0,100.0,1.0,...|            0.47|
|[290.0,104.0,4.0,...|            0.45|
|[293.0,97.0,2.0,2...|            0.64|
|[294.0,93.0,1.0,1...|            0.46|
|[294.0,95.0,1.0,1...|            0.49|
|[295.0,93.0,1.0,2...|            0.46|
|[295.0,96.0,2.0,1...|            0.47|
|[295.0,99.0,2.0,2...|            0.57|
|[296.0,99.0,2.0,3...|            0.47|
|[296.0,101.0,1.0,...|             0.6|
|[297.0,96.0,2.0,2...|            0.43|
|[297.0,96.0,2.0,2...|            0.34|
|[297.0,98.0,2.0,2...|            0.59|
|[297.0,99.0,4.0,3...|            0.54|
|[297.0,101.0,3.0,...|            0.57|
|[298.0,92.0,1.0,2...|            0.51|
|[298.0,97.0,2.0,2...|            0.45|
|[298.0,98.0,2.0,1...|            0.44|
|[298.0,99.0,1.0,1...|            0.53|
|[298.0,99.0,2.0,4...|            0.46|
+--------------------+----------------+
only showing top 20 rows

+-------------

In [84]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit ')
linear_model = lin_reg.fit(train_df)


24/09/27 01:20:14 WARN Instrumentation: [b928f2aa] regParam is zero, which might cause numerical instability and overfitting.


In [85]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.002340622269278282,0.0026498371025821667,0.004969790477750323,0.003191139828103571,0.01654263680641741,0.11710541247885004,0.01645541680556034]
Intercept: -1.4007993988530205


In [87]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.059427
r2: 0.827019


In [89]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit ","features").show()

+------------------+----------------+--------------------+
|        prediction|Chance of Admit |            features|
+------------------+----------------+--------------------+
|0.4746720616506619|            0.37|[295.0,99.0,1.0,2...|
|0.5287689798024866|            0.69|[295.0,101.0,2.0,...|
|0.4957878140801757|            0.44|[296.0,95.0,2.0,3...|
|0.5102927957854271|            0.49|[296.0,97.0,2.0,1...|
|0.5539891194854356|            0.61|[296.0,99.0,2.0,2...|
|0.5273236689645442|            0.52|[297.0,100.0,1.0,...|
|0.5690785550667741|            0.34|[298.0,98.0,2.0,4...|
|0.5981909395556371|            0.58|[298.0,100.0,3.0,...|
|0.5325997067822177|            0.54|[298.0,101.0,2.0,...|
|0.5308338325750104|            0.59|[299.0,100.0,1.0,...|
|0.5362281678789342|            0.51|[299.0,100.0,2.0,...|
| 0.564233262916664|            0.68|[299.0,100.0,2.0,...|
|0.5575927563086762|            0.63|[299.0,100.0,3.0,...|
| 0.652489500178651|            0.64|[299.0,106.0,2.0,..

In [90]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="Chance of Admit ",metricName='r2')
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.807457579521907


In [91]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit ",metricName="r2")
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.807457579521907


## Random Forest Regressor

In [105]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [106]:
featureIndexer = featureIndexer.transform(output)

In [108]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit ")
new_indexed_data.show()

+--------------------+----------------+
|     indexedFeatures|Chance of Admit |
+--------------------+----------------+
|[337.0,118.0,4.0,...|            0.92|
|[324.0,107.0,4.0,...|            0.76|
|[316.0,104.0,3.0,...|            0.72|
|[322.0,110.0,3.0,...|             0.8|
|[314.0,103.0,2.0,...|            0.65|
|[330.0,115.0,5.0,...|             0.9|
|[321.0,109.0,3.0,...|            0.75|
|[308.0,101.0,2.0,...|            0.68|
|[302.0,102.0,1.0,...|             0.5|
|[323.0,108.0,3.0,...|            0.45|
|[325.0,106.0,3.0,...|            0.52|
|[327.0,111.0,4.0,...|            0.84|
|[328.0,112.0,4.0,...|            0.78|
|[307.0,109.0,3.0,...|            0.62|
|[311.0,104.0,3.0,...|            0.61|
|[314.0,105.0,3.0,...|            0.54|
|[317.0,107.0,3.0,...|            0.66|
|[319.0,106.0,3.0,...|            0.65|
|[318.0,110.0,3.0,...|            0.63|
|[303.0,102.0,3.0,...|            0.62|
+--------------------+----------------+
only showing top 20 rows



In [109]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [110]:
training.show()

+--------------------+----------------+
|     indexedFeatures|Chance of Admit |
+--------------------+----------------+
|[290.0,100.0,1.0,...|            0.47|
|[290.0,104.0,4.0,...|            0.45|
|[293.0,97.0,2.0,2...|            0.64|
|[294.0,93.0,1.0,1...|            0.46|
|[294.0,95.0,1.0,1...|            0.49|
|[295.0,93.0,1.0,2...|            0.46|
|[295.0,99.0,1.0,2...|            0.37|
|[295.0,99.0,2.0,2...|            0.57|
|[295.0,101.0,2.0,...|            0.69|
|[296.0,95.0,2.0,3...|            0.44|
|[296.0,99.0,2.0,2...|            0.61|
|[296.0,99.0,2.0,3...|            0.47|
|[296.0,101.0,1.0,...|             0.6|
|[297.0,96.0,2.0,2...|            0.34|
|[297.0,100.0,1.0,...|            0.52|
|[297.0,101.0,3.0,...|            0.57|
|[298.0,97.0,2.0,2...|            0.45|
|[298.0,98.0,2.0,4...|            0.34|
|[298.0,99.0,1.0,1...|            0.53|
|[298.0,101.0,2.0,...|            0.54|
+--------------------+----------------+
only showing top 20 rows



In [111]:
test.show()

+--------------------+----------------+
|     indexedFeatures|Chance of Admit |
+--------------------+----------------+
|[295.0,96.0,2.0,1...|            0.47|
|[296.0,97.0,2.0,1...|            0.49|
|[297.0,96.0,2.0,2...|            0.43|
|[297.0,98.0,2.0,2...|            0.59|
|[297.0,99.0,4.0,3...|            0.54|
|[298.0,92.0,1.0,2...|            0.51|
|[298.0,98.0,2.0,1...|            0.44|
|[298.0,99.0,2.0,4...|            0.46|
|[298.0,100.0,3.0,...|            0.58|
|[298.0,105.0,3.0,...|            0.69|
|[299.0,94.0,1.0,1...|            0.42|
|[299.0,97.0,3.0,5...|            0.38|
|[300.0,99.0,1.0,1...|            0.58|
|[300.0,100.0,3.0,...|            0.62|
|[300.0,102.0,3.0,...|            0.63|
|[301.0,99.0,2.0,3...|            0.64|
|[301.0,104.0,3.0,...|            0.68|
|[301.0,106.0,4.0,...|            0.57|
|[302.0,102.0,1.0,...|             0.5|
|[302.0,102.0,3.0,...|            0.65|
+--------------------+----------------+
only showing top 20 rows



In [112]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit " )

In [114]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(training)

In [115]:
# Make predictions.
predictions = model.transform(test)

In [116]:
predictions.show()

+--------------------+----------------+-------------------+
|     indexedFeatures|Chance of Admit |         prediction|
+--------------------+----------------+-------------------+
|[295.0,96.0,2.0,1...|            0.47| 0.4590550186660273|
|[296.0,97.0,2.0,1...|            0.49| 0.5254360635158737|
|[297.0,96.0,2.0,2...|            0.43|  0.540430705120381|
|[297.0,98.0,2.0,2...|            0.59| 0.5199240121945102|
|[297.0,99.0,4.0,3...|            0.54|  0.567643205682866|
|[298.0,92.0,1.0,2...|            0.51| 0.5343580769164088|
|[298.0,98.0,2.0,1...|            0.44|0.49678202877990224|
|[298.0,99.0,2.0,4...|            0.46| 0.4231837493293322|
|[298.0,100.0,3.0,...|            0.58| 0.5959031817536276|
|[298.0,105.0,3.0,...|            0.69| 0.6747937840090809|
|[299.0,94.0,1.0,1...|            0.42| 0.4802391547768181|
|[299.0,97.0,3.0,5...|            0.38| 0.5472979043487081|
|[300.0,99.0,1.0,1...|            0.58|  0.564282028304626|
|[300.0,100.0,3.0,...|            0.62| 

In [118]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  0.052713483049392995


In [120]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.8642616003425612


In [121]:
random_forest_reg = RandomForestRegressor(featuresCol="features",labelCol="Chance of Admit " )

In [122]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(train_df)

In [124]:
# Make predictions.
predictions = model.transform(test_df)

In [125]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  0.06359511707471763


In [126]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.784325199165696


In [128]:
spark.stop()